In [1]:
import pandas as pd
from pathlib import Path
from aspiratio.utils.ir_search import search_ir_url

repo = Path('..').resolve()
master_path = repo / 'instrument_master.csv'
coverage_path = repo / 'coverage_table_updated.csv'
master = pd.read_csv(master_path, sep='\t')
coverage = pd.read_csv(coverage_path, sep='\t')

needs_ir = master[master['investor_relations_url'].isna() | (master['investor_relations_url'] == '')]
needs_ir[['CID', 'CompanyName']].head()

,CID,CompanyName


In [2]:
# Search IR URLs for missing companies
updated = []
for _, row in needs_ir.iterrows():
    cid = row['CID']
    name = row['CompanyName']
    print(f'Finding IR URL for {name} ({cid})')
    ir_url = search_ir_url(name) or ''
    updated.append((cid, ir_url))

for cid, ir_url in updated:
    if ir_url:
        master.loc[master['CID'] == cid, 'investor_relations_url'] = ir_url

master.to_csv(master_path, sep='\t', index=False)
master[master['CID'].isin([cid for cid, _ in updated])][['CID', 'CompanyName', 'investor_relations_url']].head()

,CID,CompanyName,investor_relations_url


In [3]:
# Sync coverage table IR URLs from master
coverage = coverage.merge(master[['CID', 'investor_relations_url']], left_on='Company_Identifier', right_on='CID', how='left')
coverage['IR_URL'] = coverage['investor_relations_url']
coverage.drop(columns=['CID', 'investor_relations_url'], inplace=True)
coverage.to_csv(coverage_path, sep='\t', index=False)
coverage[['CompanyName', 'FiscalYear', 'IR_URL']].head()

,CompanyName,FiscalYear,IR_URL
0,ABB Ltd,2019,https://global.abb/group/en/investors
1,ABB Ltd,2020,https://global.abb/group/en/investors
2,ABB Ltd,2021,https://global.abb/group/en/investors
3,ABB Ltd,2022,https://global.abb/group/en/investors
4,ABB Ltd,2023,https://global.abb/group/en/investors
